In [13]:
from copy import deepcopy
from math import inf

example_player_stats = {"hp": 10, "dmg":0, "armor":0, "mana":250}
example_boss_stats = {"hp": 13, "dmg":8, "armor":0}

def applyEffects(player_stats, boss_stats, effects):
    player_stats["armor"]=0
    effects_to_remove = []
    for i in range(len(effects)):
        effect = effects[i]
        dmg_dealt = effect["dmg"]
        armor = effect["armor"]
        hp = effect["hp"]
        mana = effect["mana"]
        boss_stats["hp"]-=dmg_dealt
        if boss_stats["hp"] <= 0:
            return "Player Wins"
        player_stats["armor"]+=armor
        player_stats["hp"]+=hp
        player_stats["mana"]+=mana
        effect["time"]-=1
        if effect["time"] == 0:
            effects_to_remove.append(i)
    effects_to_remove.reverse()
    for i in effects_to_remove:
        del effects[i]
    return "No Winner"

def castSpell(player_stats, boss_stats, effects, spell):
    if player_stats["mana"] < spell["cost"]:
        return "Boss Wins"
    for effect in effects:
        if effect["name"] == spell["name"]:
            return "Invalid Spell"
    player_stats["mana"]-=spell["cost"]
    dmg_dealt = spell["dmg"]
    armor = spell["armor"]
    hp = spell["hp"]
    mana = spell["mana"]
    time = spell["time"]
    if time == 0:
        player_stats["armor"]+=armor
        player_stats["hp"]+=hp
        player_stats["mana"]+=mana
        boss_stats["hp"]-=dmg_dealt
        if boss_stats["hp"] <= 0:
            return "Player Wins"
    else:
        effects.append(deepcopy(spell))
    return "No Winner"

def playerTurn(player_stats, boss_stats, effects, spell):
    result = applyEffects(player_stats, boss_stats, effects)
    if result != "No Winner":
        return result, False
    result = castSpell(player_stats, boss_stats, effects, spell)
    return result, result not in ["Boss Wins", "Invalid Spell"]

def bossTurn(player_stats, boss_stats, effects):
    result = applyEffects(player_stats, boss_stats, effects)
    if result != "No Winner":
        return result
    dmg_dealt = max(1, boss_stats["dmg"]-player_stats["armor"])
    player_stats["hp"]-=dmg_dealt
    if player_stats["hp"] <= 0:
        return "Boss Wins"
    else:
        return "No Winner"

def fillSpells():
    spells_file = open("spells.txt", 'r')
    spells = []
    for line in spells_file:
        if line.startswith("Spell"):
            continue
        desc = line.split(" ")
        name = desc[0]
        cost = int(desc[1])
        dmg = int(desc[2])
        armor = int(desc[3])
        hp = int(desc[4])
        mana = int(desc[5])
        time = int(desc[6])
        spells.append({"name": name, "cost": cost, "dmg": dmg, "armor": armor, "hp": hp, "mana": mana, "time": time})
    return spells

def fillBossStats():
    stats_file = open("stats.txt", 'r')
    boss_stats = {"hp": 0, "dmg":0, "armor":0}
    for line in stats_file:
        if line.startswith("Hit Points"):
            boss_stats["hp"]=int(line.replace(' ', '').replace('\n', '').split(':')[1])
        if line.startswith("Damage"):
            boss_stats["dmg"]=int(line.replace(' ', '').replace('\n', '').split(':')[1])
    return boss_stats

def insertStrategy(strategy, strategies):
    i = 0
    while i < len(strategies) and strategy[0] < strategies[i][0]:
        i+=1
    if i < len(strategies) and strategies[i] == strategy:
        return
    strategies.insert(i, strategy)

def playGames(starting_player_stats, starting_boss_stats, spells, hard = False):
    strategies = []
    strategies.append((0, starting_player_stats, starting_boss_stats, []))
    min_mana = inf
    while len(strategies) > 0:
        strategy = strategies.pop()
        for spell in spells:
            spent_mana = strategy[0]
            effects = deepcopy(strategy[3])
            player_stats = deepcopy(strategy[1])
            boss_stats = deepcopy(strategy[2])
            if hard:
                player_stats["hp"]-=1
                if player_stats["hp"] <= 0:
                    continue
            result, spell_was_cast = playerTurn(player_stats, boss_stats, effects, spell)
            if result == "Invalid Spell" or result == "Boss Wins":
                continue
            if spell_was_cast:
                spent_mana+=spell["cost"]
            if result == "Player Wins":
                min_mana = min(min_mana, spent_mana)
            result = bossTurn(player_stats, boss_stats, effects)
            if result == "Player Wins":
                min_mana = min(min_mana, spent_mana)
            if result == "No Winner" and spent_mana <= min_mana:
                new_strategy = (spent_mana, player_stats, boss_stats, effects)
                insertStrategy(new_strategy, strategies)
    return min_mana

def part1(starting_player_stats):
    spells = fillSpells()
    starting_boss_stats = fillBossStats()
    spent_mana = playGames(starting_player_stats, starting_boss_stats, spells)
    print(spent_mana)

def part2(starting_player_stats):
    spells = fillSpells()
    starting_boss_stats = fillBossStats()
    spent_mana = playGames(starting_player_stats, starting_boss_stats, spells, hard=True)
    print(spent_mana)

starting_player_stats = {"hp": 50, "dmg":0, "armor":0, "mana":500}
part1(starting_player_stats)
part2(starting_player_stats)

953
1289
